In [140]:
using Pkg
ENV["PYTHON"] = "/Users/aaron/miniconda3/envs/QOC/bin/python"
Pkg.build("PyCall")
using PyCall

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/1fc929f47d7c151c839c5fc1375929766fb8edcc/build.log`


In [141]:
using SparseArrays

In [142]:
@pyimport qutip as q
@pyimport scipy.sparse as sp

In [143]:
py"""
from qutip import tensor, basis, sigmax, sigmay, sigmaz
"""

In [144]:
up = q.basis(2, 0)
down = q.basis(2,1)

PyObject Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]

In [145]:
two_qubits = q.tensor(up, down)

PyObject Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

In [146]:
two_qubits.data

PyObject <4x1 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Row format>

In [147]:
@pyimport numpy as np

In [148]:
two_qubits.data.indptr

5-element Vector{Int32}:
 0
 0
 1
 1
 1

In [149]:
function py_sparse_matrix_to_julia(A)
    A = sp.csc_matrix(A)
    m, n = A.shape
    colptr = A.indptr .+ 1 
    rowval = A.indices .+ 1    
    nzval = A.data
    return SparseMatrixCSC(m, n, colptr, rowval, nzval)
end

py_sparse_matrix_to_julia (generic function with 1 method)

In [150]:
py_sparse_matrix_to_julia(two_qubits.data)

4×1 SparseMatrixCSC{ComplexF64, Int64} with 1 stored entry:
      ⋅     
 1.0 + 0.0im
      ⋅     
      ⋅     

In [151]:
qutip.expect(qutip.tensor(qutip.qeye(2), qutip.sigmaz()),two_qubits)

-1.0

In [166]:
H_drift = q.fileio.qload("../qutip_saved_objects/single_fluxonium_qubit/H_drift.qu")
H_drive = q.fileio.qload("../qutip_saved_objects/single_fluxonium_qubit/H_drive.qu")


PyObject Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

In [167]:
H_drive.data


PyObject <2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

In [168]:
H = py_sparse_matrix_to_julia(H_drive.data)

2×2 SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
     ⋅      1.0+0.0im
 1.0+0.0im      ⋅    

In [169]:
using StaticArrays

In [170]:
sm = SMatrix{2,2}(H)

2×2 SMatrix{2, 2, ComplexF64, 4} with indices SOneTo(2)×SOneTo(2):
 0.0+0.0im  1.0+0.0im
 1.0+0.0im  0.0+0.0im